# **Baggin Classifier**

In [1]:
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split


In [2]:
X, y = make_classification(n_samples=10000, n_features=10, n_informative=3)

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)
print(f'Decision Tree Accuracy : {accuracy_score(y_test, y_pred)}')

Decision Tree Accuracy : 0.875


### **Bagging**

In [11]:
bag = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=0.5,
    bootstrap=True,
    random_state=42
)

In [12]:
bag.fit(X_train, y_train)

BaggingClassifier(estimator=DecisionTreeClassifier(), max_samples=0.5,
                  n_estimators=500, random_state=42)

In [13]:
y_pred = bag.predict(X_test)

In [14]:
accuracy_score(y_test, y_pred)

0.9145

In [15]:
bag.estimators_samples_[0]

array([2523, 3113, 7114, ..., 5668, 7066, 3677])

In [16]:
bag.estimators_features_[0].shape

(10,)

### **Bagging using SVM**

In [17]:
bag = BaggingClassifier(
    estimator=SVC(),
    n_estimators=500,
    max_samples=0.25,
    bootstrap=True,
    random_state=42,
    n_jobs=-1
)

In [18]:
bag.fit(X_train, y_train)
y_pred = bag.predict(X_test)

In [19]:
print(f'Bagging using SVM: {accuracy_score(y_test, y_pred)}')

Bagging using SVM: 0.8335


### **Pasting**

In [20]:
bag = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=0.25,
    bootstrap=42,
    verbose=1,
    n_jobs=-1
)

In [21]:
bag.fit(X_train, y_train)
y_pred = bag.predict(X_test)
print(f'Pasting classifier : {accuracy_score(y_test, y_pred)}')

/usr/local/lib/python3.10/dist-packages/sklearn/utils/_param_validation.py:558: FutureWarning: Passing an int for a boolean parameter is deprecated in version 1.2 and won't be supported anymore in version 1.4.
  warnings.warn(
[Parallel(n_jobs=96)]: Using backend LokyBackend with 96 concurrent workers.
[Parallel(n_jobs=96)]: Done   2 out of  96 | elapsed:    0.2s remaining:    8.2s


Pasting classifier : 0.913


[Parallel(n_jobs=96)]: Done  96 out of  96 | elapsed:    0.3s finished
[Parallel(n_jobs=96)]: Using backend LokyBackend with 96 concurrent workers.
[Parallel(n_jobs=96)]: Done   2 out of  96 | elapsed:    0.0s remaining:    0.7s
[Parallel(n_jobs=96)]: Done  96 out of  96 | elapsed:    0.2s finished


### **Random Subspaces**

In [22]:
bag = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=1.0,
    bootstrap=False,
    max_features=0.5,
    bootstrap_features=True,
    random_state=42
)

In [23]:
bag.fit(X_train, y_train)
y_pred = bag.predict(X_test)
print(f'Random Subspaces classifier: {accuracy_score(y_test, y_pred)}')

Random Subspaces classifier: 0.9055


In [24]:
bag.estimators_samples_[0].shape

(8000,)

In [25]:
bag.estimators_features_[0].shape

(5,)

### **Random Patches**

In [26]:
bag = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=0.25,
    bootstrap=True,
    max_features=0.5,
    bootstrap_features=True,
    random_state=42
)

In [27]:
bag.fit(X_train, y_train)
y_pred = bag.predict(X_test)
print(f'Random Pathces classifier: {accuracy_score(y_test, y_pred)}')

Random Pathces classifier: 0.898


### **OOB Score**

In [28]:
bag = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=0.25,
    bootstrap=True,
    oob_score=True,
    random_state=42
)

In [29]:
bag.fit(X_train, y_train)

BaggingClassifier(estimator=DecisionTreeClassifier(), max_samples=0.25,
                  n_estimators=500, oob_score=True, random_state=42)

In [30]:
bag.oob_score_

0.915625

In [31]:
y_pred = bag.predict(X_test)
print(f'Accuracy : {accuracy_score(y_test, y_pred)}')

Accuracy : 0.9115


# Bagging Tips

- Bagging generally gives better results than Pasting
- Good results come around the 25% to 50% row sampling mark
- Random patches and subspaces should be used while dealing with high dimensional data
- To find the correct hyperparameter values we can do GridSearchCV/RandomSearchCV

### **Applying GridSearchCV**

In [5]:
from sklearn.model_selection import GridSearchCV

In [6]:
parameters = {
    'n_estimators':[50, 100, 500],
    'max_samples':[0.1, 0.4, 0.7, 1.0],
    'bootstrap':[True, False],
    'max_features':[0.1,0.4,0.7,1.0]
}

In [7]:
search = GridSearchCV(BaggingClassifier(), param_grid=parameters, cv=5, n_jobs=-1)

In [8]:
search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=BaggingClassifier(), n_jobs=-1,
             param_grid={'bootstrap': [True, False],
                         'max_features': [0.1, 0.4, 0.7, 1.0],
                         'max_samples': [0.1, 0.4, 0.7, 1.0],
                         'n_estimators': [50, 100, 500]})

In [9]:
search.best_params_

{'bootstrap': True,
 'max_features': 0.7,
 'max_samples': 1.0,
 'n_estimators': 100}

In [10]:
search.best_score_

0.919875